# Developing a Loan Scoring Algorithm for "Prêt à dépenser"

### Notebook by [Nasr-edine DRAI](https://www.hackerrank.com/d_nasredine)



### [Openclassrooms](https://openclassrooms.com/en/)

## Introduction

In this project, you are a Data Scientist working at "Prêt à dépenser", a financial company that offers consumer credits to people with little or no loan history. The company wants to implement a credit scoring tool that calculates the probability of a customer repaying a loan and then classifies the loan application as approved or rejected. The goal is to develop a classification algorithm to help decide if a loan can be granted to a customer.

## The problem domain

The customer relationship managers will be the users of the scoring tool. As they interact with customers, they need your model to be easily interpretable. The relationship managers also want a measure of the importance of the variables that led the model to give a particular probability to a customer.

<img src="../imgs/french_public_health_agency.png" />

## Verify Python Virtual Environments

#### Check the Version of the Python Interpreter

In [1]:
!python --version

Python 3.10.1


#### Verify that I'm using the right virtual environment

In [2]:
!pip -V

pip 23.0 from /Users/drainasr-edine/github/ingenieur_ia/P4_drai_nasr-edine/.venv/lib/python3.10/site-packages/pip (python 3.10)


#### Check Installed Modules in Python

Run through this notebook to make sure my environment is properly setup. Be sure to launch Jupyter from inside the virtual environment.

In [1]:
import os, sys

parent = os.path.abspath('..')
sys.path.insert(1, parent)
print(parent)

/Users/drainasr-edine/github/ingenieur_ia/P4_drai_nasr-edine


In [ ]:
from src.check_environment import run_checks

This code allows me to import modules from the parent directory in my notebook

In [2]:
from src.check_environment import run_checks
run_checks()

Using Python in /Users/drainasr-edine/github/ingenieur_ia/P4_drai_nasr-edine/.venv:
[ OK ] Python is version 3.10.1 (v3.10.1:2cd268a3a9, Dec  6 2021, 14:28:59) [Clang 13.0.0 (clang-1300.0.29.3)]

[ OK ] jupyterlab
[ OK ] jupyterlab_git
[ OK ] matplotlib
[ OK ] numpy
[ OK ] pandas
[ OK ] seaborn
[ OK ] statsmodels
[ OK ] plotly
[ OK ] colorama
[ OK ] sklearn
[ OK ] missingno
[ OK ] wordcloud


## Import Python library for data science

In [7]:
# NumPy library for numerical computing
import numpy as np

# Import the statsmodels library for statistical analysis and modeling
import statsmodels.api as sm

# Pandas library for data manipulation and analysis
import pandas as pd

# Matplotlib library for data visualization
import matplotlib.pyplot as plt

# Seaborn library for data visualization based on Matplotlib
import seaborn as sns

# Scikit-Learn library for machine learning
# import sklearn

# Tensorflow library for building and training machine learning models
# import tensorflow as tf

# Wordcloud library for generating word clouds from text data
# from wordcloud import WordCloud

### Display CSV Files with Their Sizes in a Pandas DataFrame

In [53]:
import os
import pandas as pd

folder_path = '../data'
file_list = os.listdir(folder_path)

file_details = []
for file_name in file_list:
    if file_name.endswith(".csv"):
        file_path = os.path.join(folder_path, file_name)
        file_size = os.path.getsize(file_path)
        file_details.append([file_name, file_size/10**6])

df_csv_files = pd.DataFrame(file_details, columns=["Name", "Size (MB)"])
df_csv_files.sort_values("Size (MB)", axis=0, ascending=True, inplace=True)
df_csv_files


,Name,Size (MB)
1,HomeCredit_columns_description.csv,0.037383
9,sample_submission.csv,0.536202
0,application_test.csv,26.567651
5,application_train.csv,166.133370
6,bureau.csv,170.016717
8,bureau_balance.csv,375.592889
2,POS_CASH_balance.csv,392.703158
7,previous_application.csv,404.973293
3,credit_card_balance.csv,424.582605
4,installments_payments.csv,723.118349


### Importing and Previewing CSV Data Files with Pandas

In [63]:
import pandas as pd

header = ["Table", "Row", "Description", "Special",]

df_homeCredit_columns_description = pd.read_csv("../data/HomeCredit_columns_description.csv", skiprows=1, names=header, index_col=0, encoding = 'unicode_escape')
df_homeCredit_columns_description.head()

,Table,Row,Description,Special
1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


In [64]:
# Import sample_submission.csv
df_sample_submission = pd.read_csv("../data/sample_submission.csv", sep=',')
df_sample_submission.head()

,SK_ID_CURR,TARGET
0,100001,0.5
1,100005,0.5
2,100013,0.5
3,100028,0.5
4,100038,0.5


In [65]:
# Import application_test.csv
df_application_test = pd.read_csv("../data/application_test.csv", sep=',')
df_application_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# Import application_train.csv
df_application_train = pd.read_csv("../data/application_train.csv", sep=',')
df_application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
# Import bureau.csv
df_bureau = pd.read_csv("../data/bureau.csv", sep=',')
df_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [68]:
# Import bureau_balance.csv
df_bureau_balance = pd.read_csv("../data/bureau_balance.csv", sep=',')
df_bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [69]:
# Import POS_CASH_balance.csv
df_POS_CASH_balance = pd.read_csv("../data/POS_CASH_balance.csv", sep=',')
df_POS_CASH_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [70]:
# Import previous_application.csv
df_previous_application = pd.read_csv("../data/previous_application.csv", sep=',')
df_previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# Import credit_card_balance.csv
df_credit_card_balance = pd.read_csv("../data/credit_card_balance.csv", sep=',')
df_credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [72]:
# Import installments_payments.csv
df_installments_payments = pd.read_csv("../data/installments_payments.csv", sep=',')
df_installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


### Creating a list of features with the `columns` attribute of a pandas DataFrame

In [7]:
# Access the column names and convert them to a list
features = df.columns.tolist()
print(features)

['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_fr', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_fr', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_fr', 'ingredients_text', 'allergens', 'allergens_fr', 'traces', 'traces_tags', 'traces_fr', 'serving_size', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_fr', 'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_tags', 'nutrition_grade_uk', 'nutrition_grade_fr', 'pnns_groups_1', 

### Classifying Features in a Data Set

In [8]:
import pandas as pd

# Create an empty dictionary
feature_classification = {
    'metadata': [],
    'product_info': [],
    'location_info': [],
    'ingredients': [],
    'nutritional_info': []
}

# Iterate through the list of features
for feature in features:
    # Check the type of information contained in the feature and add it to the appropriate list in the dictionary
    if '_t' in feature or '_datetime' in feature:
        feature_classification['metadata'].append(feature)
    elif '_name' in feature or '_tags' in feature or '_fr' in feature:
        feature_classification['product_info'].append(feature)
    elif '_places' in feature or '_code_geo' in feature or 'cities' in feature:
        feature_classification['location_info'].append(feature)
    elif 'ingredients' in feature:
        feature_classification['ingredients'].append(feature)
    elif '_100g' in feature or '_serving' in feature or '_value' in feature:
        feature_classification['nutritional_info'].append(feature)

import pandas as pd

# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(feature_classification, orient='index').transpose()

# Display the DataFrame with borders
display(df.style.set_caption('Feature Classification').set_table_styles([{'selector': '*', 'props': [('border', '1px solid black')]}]))



,metadata,product_info,location_info,ingredients,nutritional_info
0,created_t,product_name,manufacturing_places,None,energy_100g
1,created_datetime,generic_name,first_packaging_code_geo,None,energy-from-fat_100g
2,last_modified_t,categories_fr,cities,None,fat_100g
3,last_modified_datetime,labels_fr,purchase_places,None,saturated-fat_100g
4,packaging_tags,countries_fr,None,None,butyric-acid_100g
5,brands_tags,allergens_fr,None,None,caproic-acid_100g
6,categories_tags,traces_fr,None,None,caprylic-acid_100g
7,origins_tags,additives_fr,None,None,capric-acid_100g
8,manufacturing_places_tags,ingredients_from_palm_oil_n,None,None,lauric-acid_100g
9,labels_tags,ingredients_from_palm_oil,None,None,myristic-acid_100g


This code is creating a dictionary with keys representing different categories of information (metadata, product info, location info, ingredients, and nutritional info) and values that are lists of features that belong to each category. It then iterates through a list of features and categorizes each feature based on certain conditions (such as whether the feature name contains certain strings). Finally, it creates a DataFrame from the dictionary, transposes it, and displays it with borders. This can be useful for organizing and visualizing the different types of features contained in a data set.

## Reading a CSV file with pandas and displaying the data types of its features

In [9]:
df = pd.read_csv("../data/test.csv", sep="\t")
# Get the data type of each feature 
feature_types = df.dtypes 
print(feature_types)

/var/folders/2y/42rcpdbd4hn6jhp9z8pq2nz40000gn/T/ipykernel_39356/3209066495.py:1: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/test.csv", sep="\t")


code                        object
url                         object
creator                     object
created_t                   object
created_datetime            object
                            ...   
carbon-footprint_100g      float64
nutrition-score-fr_100g    float64
nutrition-score-uk_100g    float64
glycemic-index_100g        float64
water-hardness_100g        float64
Length: 162, dtype: object


<div class="alert alert-warning">
The warning indicates that a data file or dataset contains columns with mixed data types, which can cause problems when working with the data. The warning suggests specifying the data type for each column or setting the "low_memory" option to "False" to avoid potential problems.
</div>

## Distinguishing between quantitative and qualitative features

In [10]:
import numpy as np 
# Create an empty list to store the feature names 
quantitative_features = [] 
qualitative_features = [] 
# Iterate over the feature types 
for feature, dtype in feature_types.items():     
    # If the data type is float or int, it is quantitative     
    if dtype == float or dtype == int:         
        quantitative_features.append(feature)     
    else:         
        qualitative_features.append(feature) 

If the data type is `float` or `int`, the feature is considered quantitative and its name is added to the `quantitative_features` list. Otherwise, the feature is considered qualitative and it is added to the qualitative_features

### Displaying the quantitative features of a DataFrame

In [11]:
print(f'Quantitative features:\n{quantitative_features}') 

Quantitative features:
['no_nutriments', 'additives_n', 'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'nutrition_grade_uk', 'energy_100g', 'energy-from-fat_100g', 'fat_100g', 'saturated-fat_100g', 'butyric-acid_100g', 'caproic-acid_100g', 'caprylic-acid_100g', 'capric-acid_100g', 'lauric-acid_100g', 'myristic-acid_100g', 'palmitic-acid_100g', 'stearic-acid_100g', 'arachidic-acid_100g', 'behenic-acid_100g', 'lignoceric-acid_100g', 'cerotic-acid_100g', 'montanic-acid_100g', 'melissic-acid_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g', 'omega-3-fat_100g', 'alpha-linolenic-acid_100g', 'eicosapentaenoic-acid_100g', 'docosahexaenoic-acid_100g', 'omega-6-fat_100g', 'linoleic-acid_100g', 'arachidonic-acid_100g', 'gamma-linolenic-acid_100g', 'dihomo-gamma-linolenic-acid_100g', 'omega-9-fat_100g', 'oleic-acid_100g', 'elaidic-acid_100g', 'gondoic-acid_100g', 'mead-acid_100g', 'erucic-

In [12]:
# Create an empty dictionary
feature_classification2 = {
    'metadata': [],
    'product_info': [],
    'location_info': [],
    'ingredients': [],
    'nutritional_info': []
}

# Iterate through the list of features
for feature in quantitative_features:
    # Check the type of information contained in the feature and add it to the appropriate list in the dictionary
    if '_t' in feature or '_datetime' in feature:
        feature_classification2['metadata'].append(feature)
    elif '_name' in feature or '_tags' in feature or '_fr' in feature:
        feature_classification2['product_info'].append(feature)
    elif '_places' in feature or '_code_geo' in feature or 'cities' in feature:
        feature_classification2['location_info'].append(feature)
    elif 'ingredients' in feature:
        feature_classification2['ingredients'].append(feature)
    elif '_100g' in feature or '_serving' in feature or '_value' in feature:
        feature_classification2['nutritional_info'].append(feature)

import pandas as pd

# Create a DataFrame from the dictionary
df_quantitative_features = pd.DataFrame.from_dict(feature_classification2, orient='index').transpose()

# Display the DataFrame with borders
display(df_quantitative_features.style.set_caption('Feature Classification').set_table_styles([{'selector': '*', 'props': [('border', '1px solid black')]}]))


,metadata,product_info,location_info,ingredients,nutritional_info
0,ingredients_that_may_be_from_palm_oil_n,ingredients_from_palm_oil_n,None,None,energy_100g
1,ingredients_that_may_be_from_palm_oil,ingredients_from_palm_oil,None,None,energy-from-fat_100g
2,None,None,None,None,fat_100g
3,None,None,None,None,saturated-fat_100g
4,None,None,None,None,butyric-acid_100g
5,None,None,None,None,caproic-acid_100g
6,None,None,None,None,caprylic-acid_100g
7,None,None,None,None,capric-acid_100g
8,None,None,None,None,lauric-acid_100g
9,None,None,None,None,myristic-acid_100g


### Displaying the qualitative features of a DataFrame

In [13]:
print(f'Qualitative features:\n{qualitative_features}')

Qualitative features:
['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_fr', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_fr', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_fr', 'ingredients_text', 'allergens', 'allergens_fr', 'traces', 'traces_tags', 'traces_fr', 'serving_size', 'additives', 'additives_tags', 'additives_fr', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_tags', 'nutrition_grade_fr', 'pnns_groups_1', 'pnns_groups_2', 'states', 'states_tags', 'states_fr', 'main_category', 'main_category_fr', 'image_url', 'image_small_url']


In [14]:
# Create an empty dictionary
feature_classification3 = {
    'metadata': [],
    'product_info': [],
    'location_info': [],
    'ingredients': [],
    'nutritional_info': []
}

# Iterate through the list of features
for feature in qualitative_features:
    # Check the type of information contained in the feature and add it to the appropriate list in the dictionary
    if '_t' in feature or '_datetime' in feature:
        feature_classification3['metadata'].append(feature)
    elif '_name' in feature or '_tags' in feature or '_fr' in feature:
        feature_classification3['product_info'].append(feature)
    elif '_places' in feature or '_code_geo' in feature or 'cities' in feature:
        feature_classification3['location_info'].append(feature)
    elif 'ingredients' in feature:
        feature_classification3['ingredients'].append(feature)
    elif '_100g' in feature or '_serving' in feature or '_value' in feature:
        feature_classification3['nutritional_info'].append(feature)

# Create a DataFrame from the dictionary
df_qualitative_features = pd.DataFrame.from_dict(feature_classification3, orient='index').transpose()

# Display the DataFrame with borders
display(df_qualitative_features.style.set_caption('Feature Classification').set_table_styles([{'selector': '*', 'props': [('border', '1px solid black')]}]))


,metadata,product_info,location_info,ingredients,nutritional_info
0,created_t,product_name,manufacturing_places,None,None
1,created_datetime,generic_name,first_packaging_code_geo,None,None
2,last_modified_t,categories_fr,cities,None,None
3,last_modified_datetime,labels_fr,purchase_places,None,None
4,packaging_tags,countries_fr,None,None,None
5,brands_tags,allergens_fr,None,None,None
6,categories_tags,traces_fr,None,None,None
7,origins_tags,additives_fr,None,None,None
8,manufacturing_places_tags,nutrition_grade_fr,None,None,None
9,labels_tags,states_fr,None,None,None


### Classifying Datetime Columns as Quantitative Features

In [15]:
time_features = [item for item in qualitative_features if item.endswith(('_t', '_datetime'))]
quantitative_features.extend(time_features)
qualitative_features = [item for item in qualitative_features if item not in time_features]

A datetime column is typically considered a quantitative feature, since it represents a measurable quantity (the time at which an event occurred). In statistics and data analysis, quantitative features are those that represent numerical data, as opposed to categorical or qualitative data.

## Analyzing Feature Fill Rates in a DataFrame

In [16]:
def get_fill_rates(df):
    # Calculate the fill rate for each column
    fill_rates = df.count() / df.shape[0]
    
    # Convert the fill rates into a dataframe
    fill_rates_df = pd.DataFrame(fill_rates, columns=['Fill Rate'])
    
    # Sort the dataframe by the Fill Rate column in descending order
    fill_rates_df = fill_rates_df.sort_values(by='Fill Rate', ascending=False)
    
    return fill_rates_df
fill_rates_df = get_fill_rates(df)
fill_rates_df.style.format({'Fill Rate': '{:.2%}'}).background_gradient(cmap='RdYlGn')


,Fill Rate
last_modified_t,100.00%
last_modified_datetime,100.00%
creator,100.00%
created_t,100.00%
created_datetime,100.00%
code,99.99%
url,99.99%
states_fr,99.99%
states_tags,99.99%
states,99.99%


In [28]:
# Calculate the fill rate for each column

fill_rate = df.count() / df.shape[0]

# Apply a threshold to the fill rate and count the number of columns above the threshold
threshold = 0.6
low_fill_rate = fill_rate[fill_rate < threshold].count()

# Calculate the proportion of columns with a fill rate above the threshold
low_fill_rate_proportion = low_fill_rate / len(fill_rate)
print(len(fill_rate), low_fill_rate, low_fill_rate_proportion)
# Print the results
print(f"Number of features with fill rate under {threshold:.0%}: {low_fill_rate}")
print(f"Proportion of features with fill rate under {threshold:.0%}: {low_fill_rate_proportion:.2%}")


162 128 0.7901234567901234
Number of features with fill rate under 60%: 128
Proportion of features with fill rate under 60%: 79.01%


In [30]:
# Calculate the fill rate for each column

fill_rate = df.count() / df.shape[0]

# Apply a threshold to the fill rate and count the number of columns above the threshold
threshold = 0.7
low_fill_rate = fill_rate[fill_rate < threshold].count()

# Calculate the proportion of columns with a fill rate above the threshold
low_fill_rate_proportion = low_fill_rate / len(fill_rate)
print(len(fill_rate), low_fill_rate, low_fill_rate_proportion)
# Print the results
print(f"Number of features with fill rate under {threshold:.0%}: {low_fill_rate}")
print(f"Proportion of features with fill rate under {threshold:.0%}: {low_fill_rate_proportion:.2%}")

#todo plotly cursor intereractif 

162 133 0.8209876543209876
Number of features with fill rate under 70%: 133
Proportion of features with fill rate under 70%: 82.10%


# Observations

Based on the results provided, it is clear that:

- A significant majority (<span style="color:red">**79.01%**</span>) of the features have a fill rate below 50%.

This suggests that these features may have a high number of missing or null values, which could potentially impact the accuracy of any models that are trained on the data.

In [18]:
# Select the Fill Rate column from the fill_rates_df dataframe
fill_rate = fill_rates_df['Fill Rate']

# Create a boolean mask indicating which columns have a fill rate below 50%
mask = fill_rate >= 0.5

# Select only the columns with a fill rate above 50%
filtered_df = df.loc[:, mask]

filtered_df.shape


(320772, 34)

In [19]:
fill_rates_df = get_fill_rates(filtered_df)
fill_rates_df.style.format({'Fill Rate': '{:.2%}'}).background_gradient(cmap='RdYlGn')

,Fill Rate
last_modified_t,100.00%
last_modified_datetime,100.00%
creator,100.00%
created_t,100.00%
created_datetime,100.00%
code,99.99%
url,99.99%
states_tags,99.99%
states_fr,99.99%
states,99.99%


# Nutritional Comparison Application

If I am building an application that helps users compare the nutritional values of different products, I might want to select the following features:

- `energy_100g`: Energy content per 100g
- `proteins_100g`: Protein content per 100g
- `salt_100g`: Salt content per 100g
- `sodium_100g`: Sodium content per 100g
- `sugars_100g`: Sugar content per 100g
- `fat_100g`: Fat content per 100g
- `carbohydrates_100g`: Carbohydrate content per 100g
- `saturated-fat_100g`: Saturated fat content per 100g
- `nutrition_grade_fr`: Nutrition grade according to French food labeling system
- `nutrition-score-fr_100g`: Nutrition score according to French food labeling system per 100g
- `nutrition-score-uk_100g`: Nutrition score according to UK food labeling system per 100g

These features provide important information about the nutritional content of the products, which would be relevant and useful for this type of application.


# Building a Product Classification Model

To build a machine learning model that classifies products based on their nutritional content or other characteristics, you could use a variety of features from the list you provided. Some possible features that might be relevant for this task include:

- `energy_100g`: The energy content of the product, in kilocalories per 100 grams.
- `fat_100g`: The fat content of the product, in grams per 100 grams.
- `saturated-fat_100g`: The saturated fat content of the product, in grams per 100 grams.
- `carbohydrates_100g`: The carbohydrate content of the product, in grams per 100 grams.
- `sugars_100g`: The sugar content of the product, in grams per 100 grams.
- `fiber_100g`: The fiber content of the product, in grams per 100 grams.
- `proteins_100g`: The protein content of the product, in grams per 100 grams.
- `salt_100g`: The salt content of the product, in grams per 100 grams.
- `sodium_100g`: The sodium content of the product, in milligrams per 100 grams.
- `nutrition_grade_fr`: The nutritional grade of the product according to the French system, where "A" is the best grade and "E" is the worst.

I could also consider using additional features that provide information about the product, such as `product_name`, `brands`, `ingredients_text`, and `serving_size`, as these might be relevant for determining the overall "healthiness" of a product.


In [20]:
# Find duplicate rows
duplicate_rows = df[df.duplicated(subset=["code"])]

# Print the duplicate rows
# print(duplicate_rows)

# Count the number of duplicate rows
num_duplicates = duplicate_rows.sum()

print(f'Number of duplicate rows: {num_duplicates}')


Number of duplicate rows: code                                                           25771828648715
creator                     usda-ndb-importtacinteopenfoodfacts-contributo...
last_modified_datetime      2017-03-09T13:14:18Z2015-06-27T12:03:12Z2017-0...
first_packaging_code_geo                                                    0
countries                   USen:FRFrance,UKFranceen:USFrance,Suisse,États...
                                                  ...                        
carbon-footprint_100g                                                     0.0
nutrition-score-fr_100g                                                 615.0
nutrition-score-uk_100g                                                 603.0
glycemic-index_100g                                                       0.0
water-hardness_100g                                                       0.0
Length: 114, dtype: object


/var/folders/2y/42rcpdbd4hn6jhp9z8pq2nz40000gn/T/ipykernel_39356/1949684936.py:8: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  num_duplicates = duplicate_rows.sum()
